In [1]:
# Auto-reload internal modules
%load_ext autoreload
%autoreload 2

# External modules
import numpy as np

# Internal modules
from cosmotd.domain_walls import run_domain_wall_simulation
from cosmotd.cosmic_strings import run_cosmic_string_simulation
from cosmotd.charged_domain_walls import run_charged_domain_walls_simulation
from cosmotd.pentavac import run_pentavac_simulation
from cosmotd.plot import *
from cosmotd.parameters import *

In [10]:

# Size of simulation grid
N = 50
# Model parameters
beta = 1
eta_phi = 1
eta_sigma = np.sqrt(3) / 2
lam_phi = 2
lam_sigma = 2
charge_density = 0.1
# RNG seed
seed = 23213241
# Run time
run_time = None
# Plotter
plotter = MplMp4Plotter

In [3]:
# Domain walls simulation
run_domain_wall_simulation(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, plotter, seed, run_time)

100%|██████████| 250/250 [00:00<00:00, 1602.17it/s]


In [4]:
# Cosmic strings simulation
run_cosmic_string_simulation(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, plotter, seed, run_time)

100%|██████████| 250/250 [00:01<00:00, 199.50it/s]


In [5]:
# Charged domain walls simulation
run_charged_domain_walls_simulation(
    N, DX, DT, ALPHA_2D, beta, eta_phi, eta_sigma, lam_phi, lam_sigma, charge_density, RADIATION_ERA, plotter, seed, run_time
)

100%|██████████| 250/250 [00:00<00:00, 3622.34it/s]


In [6]:
# Pentavac model simulation with X-type and Y-type junctions
epsilon = 0.1
run_pentavac_simulation(N, 0.5, 0.1, ALPHA_2D, epsilon, RADIATION_ERA, plotter, seed, run_time)

100%|██████████| 125/125 [00:00<00:00, 142.69it/s]


In [7]:
from cosmotd.domain_walls_v2 import run_domain_wall_simulation_yield
from cosmotd.fields import calculate_energy
from cosmotd.domain_walls_v2 import potential_dw
from cosmotd.domain_wall_algorithms import find_domain_walls_convolve_diagonal

In [11]:
# Set up plotting backend
plot_api = plotter(
    PlotterSettings(
        title="Domain wall simulation", nrows=1, ncols=3, figsize=(2 * 640, 480)
    )
)
draw_settings = ImageSettings(vmin=-1.1 * eta_phi, vmax=1.1 * eta_phi, cmap="viridis")
highlight_settings = ImageSettings(vmin=-1, vmax=1, cmap="seismic")
line_settings = PlotSettings(color="#1f77b4", linestyle="-")

# Set up arrays
# Initialise wall count
iterations = list(range(int(0.5 * N * DX / DT)))
wall_count = np.empty(int(0.5 * N * DX / DT))
wall_count.fill(np.nan)
# Energies
ratios = np.empty(int(0.5 * N * DX / DT))
ratios.fill(np.nan)


sim_generator = run_domain_wall_simulation_yield(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, seed, run_time)

for i, (phi, phidot, phidotdot) in enumerate(sim_generator):
    # Find domain walls
    walls = find_domain_walls_convolve_diagonal(phi)
    wall_count[i] = np.count_nonzero(walls) / 2

    # Calculate the energy
    energy = calculate_energy(phi, phidot, potential_dw(phi, eta_phi, lam = 2 * np.pi**2 / 5**2), DX)
    masked_walls = np.ma.masked_values(walls, 0)
    ratio = np.sum(
        np.ma.masked_where(np.ma.getmask(masked_walls), energy)
    ) / np.sum(energy)
    ratios[i] = ratio

    # Plot
    plot_api.reset()
    # Real field
    plot_api.draw_image(phi, 1, draw_settings)
    plot_api.set_title(r"$\phi$", 1)
    plot_api.set_axes_labels(r"$x$", r"$y$", 1)
    # Highlight walls
    plot_api.draw_image(np.ma.masked_values(walls, 0), 2, highlight_settings)
    plot_api.set_title(r"Domain walls", 2)
    plot_api.set_axes_labels(r"$x$", r"$y$", 2)
    # Plot energy
    plot_api.draw_plot(iterations, ratios, 3, line_settings)
    plot_api.set_title("Domain wall energy", 3)
    plot_api.set_axes_labels(r"Iteration $i$", r"$\frac{H_{DW}}{H}$", 3)
    plot_api.set_axes_limits(0, int(0.5 * N * DX / DT)), 0, 1, 3)
    plot_api.flush()
plot_api.close()

100%|██████████| 250/250 [01:52<00:00,  2.22it/s]
